<a href="https://colab.research.google.com/github/ednilsonmcs/analysis-pyspark/blob/main/TrabalhoPyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [13]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
import os

In [4]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [5]:
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
!pip install -q findspark

In [7]:
import findspark 

In [14]:
findspark.init()

In [15]:
from pyspark.sql import SparkSession

In [48]:
from pyspark.sql.functions import *

In [50]:
from pyspark.sql.types import IntegerType

In [16]:
sc = SparkSession.builder.master('local[*]').getOrCreate() 

In [17]:
sc.version

'3.1.2'

In [107]:
despesasBombeiros = sc.read.options(header=True).csv(["/tmp/bombeiros"])

In [ ]:
despesasBombeiros.select(['Nome Favorecido', 'Valor Bruto (R$)', 'Valor Retido (R$)', 'Valor Pago (R$)']).sort(['Valor Pago (R$)'], ascending=False).show(n=200)

In [109]:
despesasBombeiros = despesasBombeiros.withColumn("Valor Bruto (R$)", regexp_replace("Valor Bruto (R$)", r'[,]',""))

In [110]:
despesasBombeiros = despesasBombeiros.withColumn("Valor Bruto (R$)", col("Valor Bruto (R$)").cast("numeric"))

In [111]:
despesasBombeiros = despesasBombeiros.groupBy('Nome Favorecido').sum('Valor Bruto (R$)').sort(['sum(Valor Bruto (R$))'], ascending=False)

In [114]:
despesasBombeiros.show()

+--------------------+---------------------+
|     Nome Favorecido|sum(Valor Bruto (R$))|
+--------------------+---------------------+
|BANCO DO ESTADO D...|              7436763|
|SERVICO AUTONOMO ...|              5442307|
|CIA SUL SERGIPANA...|              2374132|
|INSTITUTO NACIONA...|              1945214|
|TIFANY DA SILVA P...|              1836677|
|THIAGO DE OLIVEIR...|              1758280|
| TAIANY COSTA SANTOS|              1732218|
|VIVIANNE SANTANA ...|              1629460|
|YANTEC GRAFICA & ...|              1603803|
|EGNALDO SANTOS DA...|              1532596|
|GABRIELA OLIVEIRA...|              1518246|
|RENIELY NASCIMENT...|              1509680|
|JOSE ALEXSANDRO R...|              1505161|
|MANUELA SILVA FRE...|              1502901|
|DANIELA ALMEIDA C...|              1499641|
|W.C.S COMERCIAL D...|              1482184|
|BIANCA FREITAS DE...|              1466977|
|RAFAELA CRISTINA ...|              1447940|
| ITALO LEITE FEITOSA|              1441203|
|VANESSA R

In [112]:
type(despesasBombeiros)

pyspark.sql.dataframe.DataFrame

In [113]:
despesasBombeiros.repartition(1).write.csv("/tmp/output-despesas-bombeiros3")